# The Kaggle Rossman Competition

This was a kaggle competition to forecast sales at a pharmacy chain/dept store in Europe. It was run back in 2015.

Rossmann operates over 3,000 drug stores in 7 European countries. As a data analyst for this store chain, you are tasked with forecasting their daily sales for every store for up to six weeks in advance.

You will need to look at various factors influencing the forecast predictions - the primary ones being promotions, competition, school and state holidays, seasonality, and locality.


While working through this homework, you will:

1. see how to "grid-search" when the data is too large to use cross-validation. We will also learn how to use sklearn Pipelines to simplify the work-flow of different tranformation steps like Standardizing, One-hot encoding and Imputing missing values
2. understand some aspects of feature engineering that come in with continuous and categorical variables, and see some of the new features in sklearn 0.20
3. capture results from validation
4. learn what "categorical "embeddings" are and how they can be used to improve the performance of a multi-layer percepton



## QUESTION 1 : Preprocessing

In [ ]:
import numpy as np
import scipy.stats
import scipy.special

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
%matplotlib inline

In [ ]:
from pathlib import Path

### 1.1 Load , check and clean the data

In [ ]:
data = Path('./data')

We load the data from the 'data' folder and engage in some cleaning. A lot of cleaning of this dataset has already been done for us. Some features have been created. In particular we moved from dates to week-of-year, day-of week, etc. For example the 49th and 50th weeks of the year may have higher sales!

In [ ]:
train_df = pd.read_csv(data/"train_clean.csv.gz", compression='gzip').drop(['index', 'PromoInterval'], axis=1)
test_df = pd.read_csv(data/"test_clean.csv.gz", compression='gzip').drop(['index', 'PromoInterval'], axis=1)

In [ ]:
train_df['Events'] = train_df['Events'].fillna('None')
test_df['Events'] = test_df['Events'].fillna('None')

Plot a histogram of Sales. What do you see?

In [ ]:
# your code here



We log-transform the dependent variable because it is long-tailed, i.e., the distribution has many occurrences far from the 'central' part of the distribution

In [ ]:
train_resp = np.log(train_df['Sales'].copy())
train_df = train_df.drop('Sales', axis=1)

# We drop the 'Customers' from train_df and 'Id' from test_df as they're not present in the other dataframe
train_df = train_df.drop('Customers', axis=1)
test_df = test_df.drop('Id', axis=1)

Lets get some idea about our dataset - what are the different variables used? How do their values look like?

In [ ]:
train_df.head()

In [ ]:
train_df.shape, test_df.shape

The training and test datasets are already given to us sorted by the decreasing order of the 'date' column (the latest date being first)

In [ ]:
train_df.Date # latest date first

In [ ]:
train_df.columns

### 1.2 Types of variables and cardinality

We make a note of which variables are categorical and which are not. This is a choice. If cardinality(unique elements in a variable) is not too high, binning or categorizing can be beneficial. Often this will be true for integer valued variables.
>**Binning** is a way to group a number of more or less continuous values into a smaller number of "bins" or "categories"

In [ ]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw', 'Promo', 'SchoolHoliday']

cont_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday']

We look for missing data and store the column names where this happend in the continuous data

In [ ]:
nacols=[]
for v in cont_vars:
    if np.sum(train_df[v].isnull()) > 0:
        nacols.append(v)
        print(v, np.sum(train_df[v].isnull()))

And look at some cardinalities (unique values) of the continuous data: if we have none below 10, we won't engage in binning. 
Print all continuous variables below and also print the unique elements for those variables for which the cardinality is less than 10 

In [ ]:
# your code here




We do a similar looksie on the categorical variables. Some of these have many levels. Print all categorical variables below and also print the unique elements for those variables for which the cardinality is less than 50.  

In [ ]:
# your code here




### 1.3 Creating a validation set

The construction of a validation or "development" set is not always a `test_train_split` deal. Here we create a validation set of "latest" data, corresponding in date and size to what we have in the test set. Hopefully this will make sure we have similar distributions of features and outcomes on both.

#### Look at the train & test data and answer why we shouldn't have a random test-train split in this case

*your answer here*



In [ ]:
cut = train_df['Date'][(train_df['Date'] == train_df['Date'][len(test_df)])].index.max()
cut

In [ ]:
valid_idx = range(cut)
train_idx = list(np.setdiff1d(range(train_df.shape[0]), valid_idx))

In [ ]:
trdf = train_df.iloc[train_idx]
vadf = train_df.iloc[valid_idx]

In [ ]:
trdf.shape, vadf.shape

### 1.4 Transformation Pipelines

This is the definition of `pipeline` class according to scikit-learn is:
>Sequentially apply a list of transforms and a final estimator. Intermediate steps of pipeline must implement fit and transform methods and the final estimator only needs to implement fit.

In most ML projects, there are quite often a number of transformational steps such as encoding categorical variables, feature scaling and normalisation that need to be performed. However, in a typical machine learning workflow you will need to apply all these transformations at least twice. Scikit-learn pipelines are a tool to simplify this process.

The pipeline class allows sticking multiple processes into a single scikit-learn estimator. It has fit, predict and score method just like any other estimator (ex. LinearRegression).

#### Why Pipelines?

1. Pipelines enforce implementation and desired order of steps in your project, which in turn helps in reproducibility and creating a convenient work-flow.
2. They also prevent data leakage in your validation set during cross-validation by ensuring that data preparation like standardization is constrained to each fold of your cross validation procedure.

Ok, now we'll use the new `ColumnTransformer` with imputation, missing-data indicators, the new `OrdinalEncoder`, and the usual `StandardScaler`. The [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) writes this about `ColumnTransformer`:
>Applies transformers to columns of an array or pandas DataFrame.
This estimator allows different columns or column subsets of the input to be transformed separately and the features generated by each transformer will be concatenated to form a single feature space. This is useful for heterogeneous or columnar data, to combine several feature extraction mechanisms or transformations into a single transformer.

In [4]:
from sklearn.impute import SimpleImputer,MissingIndicator
from sklearn.pipeline import make_pipeline, make_union, Pipeline

In [ ]:
impu = SimpleImputer(strategy="median") # replaces the missing values of a column by the median

In [ ]:
mi = MissingIndicator() # create, fit, and transform a missingness indicator
mi.fit(trdf[nacols])
Xtrmi = mi.transform(trdf[nacols])
Xvami = mi.transform(vadf[nacols])

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
ss = StandardScaler()
oe = OrdinalEncoder()

In [ ]:
trdf_cat = trdf[cat_vars]
trdf_cont = trdf[cont_vars]

We construct two pipelines, one for categoricals and one for continuous variables

The first step in building the pipeline is to define each transformer type. The convention here is generally to create transformers for the different variable types. All the continuous variables need to be median imputed (wherever required) and scaled, so we create a transformer which includes a SimpleImputer to fill in any missing values and applies a StandardScaler


In [ ]:
cont_pipe = Pipeline([("imp",impu), ("scale", ss)])

#### Create a pipeline `cat_pipe` for categorical variables that should contain a transformer that can one-hot encode each variable

In [334]:
# your code here


We now combine them here in a transformer list.

In [ ]:
transformers = [('cat', cat_pipe, cat_vars),
                    ('cont', cont_pipe, cont_vars)]

Now we use a `ColumnTransformer` to combine these and fit it on train data

In [ ]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=transformers, remainder='drop')

In [ ]:
ct.fit(trdf)

In [ ]:
Xtr = ct.transform(trdf)
Xval = ct.transform(vadf)

In [ ]:
Xtr.shape, Xtrmi.shape

We concatenate the old indicators back in. The transformer lists all the categoricals first, since thats the first item in `transformers`, so we pre-pend.

In [ ]:
Xtrain = np.concatenate([Xtrmi, Xtr], axis=1)
Xtrain.shape

In [ ]:
Xvalid = np.concatenate([Xvami, Xval], axis=1)
Xvalid.shape

sklearn-pipelines lose our nice pandas names. so we bring them back.

In [ ]:
cols = trdf.columns
actcols = []
actcolcount = 0
nacols_cat = []
for k in nacols:
    actcols.append((k+'_missing', 'cont'))
    nacols_cat.append(k+'_missing')
    actcolcount+=1
for k in cat_vars+cont_vars:
    if k in cat_vars:
        actcols.append((k, "cat"))
        actcolcount+=1
    if k in cont_vars:
        actcols.append((k, "cont"))
        actcolcount+=1
        
list(enumerate(actcols)), actcolcount

## Time to learn
We will follow two approches here to forecast sales and see how they perform relative to each other:
1. Gradient Boosting Regression Trees
2. Multi-layer Perceptron Model using Entity Embeddings


## QUESTION 2. Forecast using Gradient Boosting Regression Trees(GBRT)
We need to first split the y (the log of the y, really) into **ytrain** and **yvalid**

In [335]:
# your code here



In [ ]:
ytrain.shape, yvalid.shape

and import what we need to for Gradient Boosting

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

Peter Prettenhofer, who wrote sklearns GBRT implementation writes in his pydata14 talk (worth watching!) ([link](https://www.youtube.com/watch?v=-5l3g91NZfQ) here)

>Hyperparameter tuning - I usually follow this recipe to tune the hyperparameters:

> 
- Pick n_estimators as large as (computationally) possible (e.g. 3000)
- Tune max_depth, learning_rate, min_samples_leaf, and max_features via grid search
- A lower learning_rate requires a higher number of n_estimators. Thus increase n_estimators even more and tune learning_rate again holding the other parameters fixed

>This last point is a trade-off between number of iterations or runtime against accuracy. And keep in mind that it might lead to overfitting.

Let me add however, that poor learners do rather well. So you might want to not cross-validate max_depth. And min_samples_per_leaf is not independent either, so if you do use cross-val, you might just use one of those.

### 2.1 Use Grid-search without cross-validation
We use `ParameterGrid` here to construct the entire grid for us! We put the output in a list of dictionaries and then save it in a dataframe. We might want to persist such dataframes to disk.

In [ ]:
param_grid = {'learning_rate': [0.1, 0.01],
              'max_depth': [1,2, 3],
              'max_features': [0.2, 0.6]
              }

In [ ]:
from sklearn.model_selection import ParameterGrid


Use the Gradient Boosting Regressor method **(n_estimators = 200)** to fit on your training data. Calculate the mean squared error for both the validation **('mse')** and the training set **('msetr')** for each combination of parameters in the `param_grid` and store the results along with the parameters in a list **'ds'**. [sklearn documentation link](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)

In [ ]:
ds=[]
for p in ParameterGrid(param_grid):
    print(p)
    # your code here

    
    
    
    
    

In [ ]:
dsdf = pd.DataFrame.from_records(ds)
dsdf.sort_values('mse')

### 2.2 Predict on the test data-set

We will now predict the sales on the test data-set

Use the best **GBRT** estimator and fit again on the training set

In [337]:
# your code here




Use the Column transformer defined earlier to transform the test dataset and get it in the correct format. Follow the same process that we did for Xtrain and Xvalid.  

In [338]:
# your code here




Predict the sales on the test data-set (**test_df**) and store the result in a new column **forecast_gb**

In [339]:
# your code here


We shall now write a function to plot the forecasts vs the actual sales for the training and the test data combined. To do that, we first predict the sales on the **trdf** and **vadf**, concatenate the two dataframes together, add back the actual sales column **(train_resp)** and concatenate it again with **test_df**.

In [342]:
def forecast_plot(model, store, Xtrain, Xvalid):
    trdf['forecast'] = model.predict(Xtrain)
    vadf['forecast'] = model.predict(Xvalid)
    train_df = pd.concat([vadf,trdf])
    train_df['Actual'] = train_resp
    all_data = pd.concat([test_df, train_df])
    
    plt.figure(figsize=(20,8))
    plt.plot(pd.to_datetime(all_data[all_data['Store']==store]['Date']),all_data[all_data['Store']==store]['forecast'])
    plt.plot(pd.to_datetime(all_data[all_data['Store']==store]['Date']),all_data[all_data['Store']==store]['Actual'])
    plt.title('Forecast')
    plt.ylabel('Log Sales')
    plt.legend(['forecast', 'actual'], loc='best')
    plt.show()    

Plot the actual vs forecasted sales for Store 1

In [341]:
# your code here



## QUESTION 3. A Multi-Layer Perceptron Model

This is based on the 3rd prize winning entry, whose authors wrote a [paper](https://arxiv.org/pdf/1604.06737.pdf) afterwords.

What we are first going to do is to reduce the cardinality dimensionality of our categoricals by using **embeddings**. 

### What are embeddings?
An embedding is a mapping of discrete - categorical - variable to a vector of continuous variables. Neural Network embeddings are useful because they can reduce the dimensionality of categorical variables and meaningfully represent categories in the transformed space.
This technique is often used not only in recommender systems(via matrix factorization), but also in NLP models such as `word2vec`.

In our problem, consider the variable `store` as an example. This is a categorical predictor and we usually **one-hot encode** this - a single store would be of length 1115 bit-vector with one bit flipped on.

### Problems with One-hot encoding
1. The 1115 stores will have some commonalities, but the one-hot encoding doesn't represent this. The dot-product(cosine similarity) of any 2 one-hot encoded stores will be 0
2. The store variable has high cardinality (1115 unique categories) - in this case, the dimensionality of the transformed variable becomes unmanageable

Using embeddings for `store` will enable us to learn the store 'personalities', which can then be used later in other models for sales predictions, or even for other tasks.

### Training an embedding

- Normally you would do a linear or MLP regression with sales as the target, and both continuous and categorical features
- We need to replace the 1-hot encoded categorical features by "lower-width" embedding features.
- This is equivalent to considering a neural network with the output of an additional **Embedding Layer** concatenated in
- The Embedding Layer is simply a Linear Regression

![](./images/embmlp.png)

Here we divide the cardinality by 2 and add 1 to get the no. of embedding features (this is a heuristic). If the cardinality is high, we clamp the size of the latent space down at 50.

In [ ]:
cards={}
for k in nacols_cat:
    cards[k] = (2,2)
for k in cat_vars :
    embed_sz_base = trdf[k].unique().size//2 + 1
    embed_sz = (embed_sz_base <=50)*embed_sz_base + 50*((embed_sz_base > 50))
    cards[k] = (trdf[k].unique().size, embed_sz)
cards

### 3.1 Building the model architecture using the Keras Functional API
We have to be careful (very book-keepy) in constructing a model in Keras. We use the Keras Functional API as opposed to the Sequential API. The architecture can be summarized in the below steps:
1. 25 input layers for categorical variables which feed into an Embedding Layer and then into a Reshape Layer
2. 14 input layers for continuous variables
3. Concatenating the 25 Reshape layers and the 14 input layers
4. Connecting the concatenated layer to a 1000 unit Dense Layer, which is connected to a 500 unit Dense Layer, which is then connected to the Output - a single unit Dense Layer

In [ ]:
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding

def build_keras_model():
    input_cat = []
    output_embeddings = []
    for k in nacols_cat+cat_vars:
        print('{}_embedding'.format(k))
        input_1d = Input(shape=(1,))
        output_1d = Embedding(cards[k][0], cards[k][1], name='{}_embedding'.format(k))(input_1d)
        output = Reshape(target_shape=(cards[k][1],))(output_1d)
        input_cat.append(input_1d)
        output_embeddings.append(output)

    main_input = Input(shape=(len(cont_vars),), name='main_input')
    output_model = Concatenate()([main_input, *output_embeddings])
    output_model = Dense(1000, kernel_initializer="uniform")(output_model)
    output_model = Activation('relu')(output_model)
    output_model = Dense(500, kernel_initializer="uniform")(output_model)
    output_model = Activation('relu')(output_model)
    output_model = Dense(1)(output_model)
    kmodel = KerasModel(
        inputs=[*input_cat, main_input], 
        outputs=output_model
)
    kmodel.compile(loss='mean_squared_error', optimizer='adam')
    return kmodel

def fitmodel(kmodel, Xtr, ytr, Xval, yval, epochs, bs):
    h = kmodel.fit(Xtr, ytr, validation_data=(Xval, yval),
                       epochs=epochs, batch_size=bs)
    return h

The data input needs to match our construction:

In [ ]:
list_cat_trains=[]
list_cat_valids=[]
catlen=len(nacols_cat+cat_vars)
for i in range(catlen):
    list_cat_trains.append(Xtrain[:,i])
    list_cat_valids.append(Xvalid[:,i])
cont_train=Xtrain[:,catlen:]
cont_valid=Xvalid[:,catlen:]

### 3.2 Run the model for 10 epochs and a batch-size of 256

In [340]:
# your code here



In [ ]:
# Function to plot training vs validation loss
def plot_model_history(model_history):
    # summarize history for loss
    plt.plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    plt.plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.ylim([0,0.3])
    plt.legend(['train', 'val'], loc='best')
    plt.show()

Plot the training and validation loss across epochs

In [ ]:
# your code here


How does the Categorical Embedding model perform as opposed to the GBRT model?

We predict the forecasts on the test data-set and then compare the actuals vs forecast for the entire data (train+test)

In [1]:
# your code here


Plot the actuals vs forecasts on the entire data-set

In [2]:
# your code here
